# Script ETL o fato FLUXO
*autor: André Costa (2019-03-09)* [https://www.linkedin.com/in/a-l-costa]

- Este script executa o processo ETL para gerar a tabela de fato FLUXO
- *AVISO:* Este script substitui completamente os dados armazenados anteriormente pelos novos dados, caso haja conflito de chave única

In [1]:
from pyspark.sql.types import *
import pandas as pd
from pyspark.sql.functions import UserDefinedFunction, countDistinct

spark.sql("use geodata")

DataFrame[]

## Extração do arquivo *eventos_de_fluxo.csv*
- O arquivo lido gera um Spark Dataframe, que é imediatamente registrado no Hive com nome **raw_fluxo**
- *AVISO:* Este script não prevê descomprimir o arquivo de dados sendo carregado

In [2]:
rawFluxoSchema = StructType([
             StructField('codigo', StringType()),
             StructField('when', TimestampType()),
             StructField('concorrente_id', LongType()),
            ])
rawFluxo = spark.read.csv("file:/home/hadoop/data/eventos_de_fluxo.csv",
                          schema = rawFluxoSchema, header = True).drop('codigo')
rawFluxo.registerTempTable('raw_fluxo')
print('Contagem de registros: ', rawFluxo.count())
rawFluxo.take(3)

Contagem de registros:  248589


[Row(when=datetime.datetime(2017, 7, 27, 9, 51, 2), concorrente_id=650509405109544),
 Row(when=datetime.datetime(2017, 6, 24, 14, 0, 26, 405000), concorrente_id=650509405109544),
 Row(when=datetime.datetime(2017, 7, 6, 21, 51, 11, 56000), concorrente_id=650509405109544)]

## Geração do ID referente ao CALENDARIO
- Neste etapa, definimos uma função customizada para definir o ID do CALENDÁRIO sem a necessidade de realizar uma consulta ao banco de dados

In [3]:
BIG_BANG = pd.Timestamp('2000-01-01')
manha = pd.Timedelta('12 hours')
tarde = pd.Timedelta('18 hours')

def get_calendar_id(ts):
    elapsed_time = (ts - BIG_BANG)
    DAYS = elapsed_time.days
    intraday_timedelta = (elapsed_time - pd.Timedelta(days=DAYS))
    if (intraday_timedelta>tarde):
        periodo = 2
    elif (intraday_timedelta>manha):
        periodo = 1
    else:
        periodo = 0
    return DAYS*3 + periodo

print('ID para a data 2017-07-27 18:00:2 -->', get_calendar_id(pd.Timestamp('2017-07-27 18:00:2')))
print('ID armazenado na dimesão calendario')
spark.sql('select id FROM calendario WHERE ano=2017 and mes=7 and dia=27 and periodo=2').show()

ID para a data 2017-07-27 18:00:2 --> 19253
ID armazenado na dimesão calendario
+-----+
|   id|
+-----+
|19253|
+-----+



## Geração do ID CALENDARIO e contagem de ocorrencias repetidas
- Nesta etapa, a função de ID do calendário é aplicada, também é realizada a agregação de chaves unicas repetidas, neste caso, composta dos atributos *calendario_id* e *concorrente_id*

In [4]:
calendar_id = UserDefinedFunction(get_calendar_id, LongType())
indexedFluxo = rawFluxo.withColumn('when', calendar_id(rawFluxo.when)) \
    .groupBy('when', 'concorrente_id').count()

indexedFluxo.registerTempTable('indexed_fluxo')

print('Contagem de registros: ', indexedFluxo.count())
indexedFluxo.take(3)

Contagem de registros:  40215


[Row(when=19243, concorrente_id=557361084467209, count=1),
 Row(when=19237, concorrente_id=1637714039783369, count=1),
 Row(when=19191, concorrente_id=133752033461795, count=5)]

## Ligação com a dimensão CONCORRENTE
- Neste etapa, os eventos de fluxo são conectados à dimensão CONCORRENTE (e à mini-dimensão BAIRRO), obtendo as respectivas SK

In [5]:
spark.sql('create TEMPORARY view linked_fluxo AS \
    select a.when as calendario_id, b.id as concorrente_id, c.id as bairro_id, a.count as ocorrencias \
    FROM indexed_fluxo as a INNER join concorrente as b ON a.concorrente_id=b.codigo \
    INNER join bairro as c ON b.bairro_id=c.codigo and isnull(c.end_date)')

DataFrame[]

## Carregamento dos eventos de FLUXO
- Nesta etapa, todos os registros do warehouse são combinados com os novos registros em uma operação de *FULL JOIN*
- Então, para cada atributo, é escolhido o primeiro valor não nulo, iniciando pelos novos registros
- Finalmente, os dados são carregados em uma tabela *buffer*, a tabela original é removida, e a tabela buffer é renomeada como definitiva

In [6]:
finalFluxo = spark.sql('select NVL(a.concorrente_id, b.concorrente_id) as concorrente_id, \
    NVL(a.bairro_id, b.bairro_id) as bairro_id, \
    NVL(a.calendario_id, b.calendario_id) as calendario_id, \
    NVL(a.ocorrencias, b.ocorrencias) as ocorrencias \
    FROM linked_fluxo as a FULL join fluxo as b ON a.calendario_id=b.calendario_id \
    and a.concorrente_id=b.concorrente_id')
finalFluxo.registerTempTable('final_fluxo')

print('Contagem de registros:', finalFluxo.count())

spark.sql('create table fluxo_buff like fluxo')
spark.sql('insert into table fluxo_buff SELECT * from final_fluxo')
spark.sql('drop table fluxo')
spark.sql('alter table fluxo_buff RENAME TO fluxo')
spark.sql('select * from fluxo limit 5').show()

Contagem de registros: 40215
+--------------+----------+-------------+-----------+
|concorrente_id| bairro_id|calendario_id|ocorrencias|
+--------------+----------+-------------+-----------+
|   -1286822256|-455108858|        19102|          2|
|    1099627934|-636139672|        19105|          1|
|    -250035517| -12108643|        19145|          1|
|   -2113182782| -12108643|        19149|          1|
|     779331024|-636139672|        19155|          3|
+--------------+----------+-------------+-----------+

